<a href="https://colab.research.google.com/github/MatheusHavoc/Web_Scrapping/blob/main/mercado_livre_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#  Importar bibliotecas
from bs4 import BeautifulSoup
from bs4.element import Tag
import requests
from time import sleep
import pandas as pd
import numpy as np
from IPython.display import clear_output

In [ ]:
# Habilitar visualizador de tabelas do Google Colab
from google.colab import data_table

In [ ]:
# Acessar site 
url_base  =  'https://lista.mercadolivre.com.br/'

# - Obtendo produtos do Mercado Livre a partir de uma busca realizada pelo usuário
produto_nome  =  input ( 'Qual produto você deseja? ' ) 
ML_address  =  (url_base + produto_nome) # É interessante que as siglas tenham um complemento pra entender do que se trata
#------------------------------------------------------------------------------------------------------

ML_req = requests.get(ML_address)
soup = BeautifulSoup(ML_req.text, "html.parser") # É interessante manter as convenções adotadas nas próprias bibliotecas. No casso do BeautifulSoup, recomendam usar o nome de variável 'soup' (https://beautiful-soup-4.readthedocs.io/en/latest/#quick-start)

Qual produto você deseja? iPhone


In [ ]:
# Definir todos os CSS Selectors

# Página de resultados
css_product_card = ".ui-search-layout__item.shops__layout-item"
css_product_title = ".ui-search-item__title.shops__item-title"
css_product_details_link = ".ui-search-item__group__element.shops__items-group-details.ui-search-link"
css_product_price = ".price-tag-fraction"

# Página do produto
css_product_attributes_table_row = ".andes-table__row.ui-vpp-striped-specs__row"
css_product_attributes_color_label = ".ui-pdp-variations__label.ui-pdp-color--BLACK"
css_product_attributes_color = ".ui-pdp-variations__selected-label.ui-pdp-color--BLACK"

In [ ]:
# Selecionar todos os cards dos produtos

product_cards = soup.select(css_product_card)

In [ ]:
# Criar uma função que busca o elemento <a href="..." class="ui-search-item__group__element shops__items-group-details ui-search-link"></a> dentro do card do produto e returna uma tupla com valor de title e href

def get_product_basic_info(product_card: Tag) -> str:
  product_details_link_element = product_card.select_one(css_product_details_link)
  product_title = product_details_link_element.get_attribute_list('title')[0]
  product_link = product_details_link_element.get_attribute_list('href')[0]
  product_price = product_card.select_one(css_product_price).text
  return product_title, product_link, product_price

In [ ]:
# Criar uma função que busca todas as linhas de atributos do produto a partir da página do produto, transforma em um dicionário {nome do atributo: valor} e retorna esse dicionário

def get_product_attributes(product_soup: BeautifulSoup) -> dict:
  attributes_table_rows = product_soup.select(css_product_attributes_table_row)
  attributes_dict = {}
  for row in attributes_table_rows:
    attribute_name = row.select_one('th').text
    attribute_value = row.select_one('td').text
    attributes_dict[attribute_name] = attribute_value
  description_attribute_labels = product_soup.select(css_product_attributes_color_label)

  if len(description_attribute_labels): # Alguns atributos ficam ao lado da foto do produto (fora da tabela), verificar se foi encontrado algum e insere no dicionário de atributos
    for description_attribute_label in description_attribute_labels:
      if "Memória interna:" in description_attribute_label.text: attributes_dict['Memória interna'] = description_attribute_label.text.replace("Memória interna:", "")
      if "Cor:" in description_attribute_label.text: attributes_dict['Cor'] = description_attribute_label.text.replace("Cor:", "")

  return attributes_dict

In [ ]:
# Faz um loop na lista de cards de produtos, requisita o link do produto e armazena o título, link e atributos do produto em uma lista

products = []
for idx, product_card in enumerate(product_cards):
  product_title, product_link, product_price = get_product_basic_info(product_card)
  product_attributes = {}
  while len(product_attributes.keys()) <= 2:
    print(f'{idx}/{len(product_cards)} | Tentando buscar atributos do produto: {product_title}') # Colocando uma repetição pois as vezes a página do Mercado Livre não retorna a tabela de atributos, então é preciso tentar novamente.
    product_soup = BeautifulSoup(requests.get(product_link).text, 'html.parser')
    product_attributes = get_product_attributes(product_soup)
  products.append({
      'Título': product_title,
      'Link': product_link,
      'Preço': product_price,
      **product_attributes
  })
clear_output()

In [ ]:
df = pd.DataFrame.from_dict(products)
df['Preço'] = df['Preço'].astype('float')

In [ ]:
df

,Título,Link,Preço,Marca,Linha,Modelo,Cor,Memória interna,Memória RAM,Com ranhura para cartão de memória,...,Inclui lápis,Autonomia de conversação,Com botão SOS,É resistente a quedas,Autonomia da batería em standby,Homologação Anatel Nº,Com IMEI,Operadora,Com reconhecimento de mão,Resolução da câmera grande-angular
0,Apple iPhone 11 (64 GB) - Branco,https://www.mercadolivre.com.br/apple-iphone-1...,3.025,Apple,iPhone,iPhone 11,Branco,64 GB,4 GB,Não,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Apple iPhone 11 (256 GB) - (PRODUCT)RED,https://www.mercadolivre.com.br/apple-iphone-1...,3.549,Apple,iPhone,iPhone 11,(Product)Red,256 GB,4 GB,Não,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Apple iPhone 14 Pro (128 GB) - Dourado,https://www.mercadolivre.com.br/apple-iphone-1...,8.399,Apple,iPhone,iPhone 14 Pro,Dourado,128 GB,6 GB,NaN,...,Não,75 h,Sim,Sim,NaN,NaN,NaN,NaN,NaN,NaN
3,Apple iPhone 14 Pro Max (1 TB) - Roxo-profundo,https://www.mercadolivre.com.br/apple-iphone-1...,14.799,Apple,iPhone,iPhone 14 Pro Max,Roxo-profundo,1 TB,6 GB,NaN,...,Não,95 h,Sim,Sim,NaN,NaN,NaN,NaN,NaN,NaN
4,Apple iPhone 13 Pro (128 GB) - Dourado,https://www.mercadolivre.com.br/apple-iphone-1...,7.240,Apple,iPhone,iPhone 13 Pro,Dourado,128 GB,6 GB,Não,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Apple iPhone 12 Pro (128 GB) - Grafite,https://www.mercadolivre.com.br/apple-iphone-1...,6.999,Apple,iPhone,iPhone 12 Pro,Grafite,128 GB,6 GB,Não,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Apple iPhone XR 128 GB - Preto,https://www.mercadolivre.com.br/apple-iphone-x...,2.445,Apple,iPhone,iPhone XR,Preto,128 GB,3 GB,Não,...,NaN,25 h,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,iPhone X 64 GB cinza-espacial,https://www.mercadolivre.com.br/iphone-x-64-gb...,2.020,Apple,iPhone,iPhone X,Cinza-espacial,64 GB,3 GB,NaN,...,NaN,21 h,NaN,NaN,240 h,NaN,NaN,NaN,NaN,NaN
8,Apple iPhone 12 Pro Max (256 GB) - Grafite,https://www.mercadolivre.com.br/apple-iphone-1...,9.500,Apple,iPhone,iPhone 12 Pro Max,Grafite,256 GB,6 GB,Não,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Apple iPhone 12 (128 GB) - Branco,https://www.mercadolivre.com.br/apple-iphone-1...,4.620,Apple,iPhone,iPhone 12,Branco,128 GB,4 GB,Não,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Apagando titulo df
df = df.drop(['Título'], axis=1) 
# Apagando  colunas com valores nulls 
df = df.dropna(axis=1)  
df = df.reset_index(drop=True)
df.head(5)

,Link,Preço,Marca,Linha,Modelo,Cor,Memória interna,Memória RAM,Nome do sistema operacional,Versão original do sistema operacional,...,Tecnologia da tela,Pixels por polegada da tela,Com tela tátil,Tipo de bateria,Com bateria removível,Com teclado QWERTY físico,Modelo do processador,Com acelerômetro,Com sensor de proximidade,Com giroscópio
0,https://www.mercadolivre.com.br/apple-iphone-1...,3.025,Apple,iPhone,iPhone 11,Branco,64 GB,4 GB,iOS,13,...,IPS,326 ppi,Sim,Íon de lítio,Não,Não,Apple A13 Bionic,Sim,Sim,Sim
1,https://www.mercadolivre.com.br/apple-iphone-1...,3.549,Apple,iPhone,iPhone 11,(Product)Red,256 GB,4 GB,iOS,13,...,IPS,326 ppi,Sim,Íon de lítio,Não,Não,Apple A13 Bionic,Sim,Sim,Sim
2,https://www.mercadolivre.com.br/apple-iphone-1...,8.399,Apple,iPhone,iPhone 14 Pro,Dourado,128 GB,6 GB,iOS,16,...,OLED,460 ppi,Sim,Íon de lítio,Não,Não,Apple A16 Bionic,Sim,Sim,Sim
3,https://www.mercadolivre.com.br/apple-iphone-1...,14.799,Apple,iPhone,iPhone 14 Pro Max,Roxo-profundo,1 TB,6 GB,iOS,16,...,OLED,460 ppi,Sim,Íon de lítio,Não,Não,Apple A16 Bionic,Sim,Sim,Sim
4,https://www.mercadolivre.com.br/apple-iphone-1...,7.240,Apple,iPhone,iPhone 13 Pro,Dourado,128 GB,6 GB,iOS,15,...,OLED,460 ppi,Sim,Íon de lítio,Não,Não,Apple A15 Bionic,Sim,Sim,Sim


In [ ]:
# Mostrar o produto index 5 como exemplo

df.iloc[5].to_dict()

{'Link': 'https://www.mercadolivre.com.br/apple-iphone-12-pro-128-gb-grafite/p/MLB16163679?pdp_filters=category:MLB1055#searchVariation=MLB16163679&position=6&search_layout=stack&type=product&tracking_id=4a9dece9-e5a5-4ea7-a42d-bdaac0516fd8',
 'Preço': 6.999,
 'Marca': 'Apple',
 'Linha': 'iPhone',
 'Modelo': 'iPhone 12 Pro',
 'Cor': 'Grafite',
 'Memória interna': '128 GB',
 'Memória RAM': '6 GB',
 'Nome do sistema operacional': 'iOS',
 'Versão original do sistema operacional': '14',
 'Resolução da câmera traseira principal': '12 Mpx',
 'Resolução de vídeo da câmera traseira': '3840 px x 2160 px',
 'Resolução da câmera frontal principal': '12 Mpx',
 'Com câmera': 'Sim',
 'Quantidade de câmeras traseiras': '3',
 'Abertura do diafragma da câmera traseira': 'f 1.6/f 2.0/f 2.4',
 'Quantidade de câmeras frontais': '1',
 'Resolução de vídeo da câmera frontal': '3840 px x 2160 px',
 'Altura x Largura x Profundidade': '146.7 mm x 71.5 mm x 7.4 mm',
 'Rede': '5G',
 'Com conector USB': 'Não',
 'C